# <center>Projet Neo4j - PapersWithCode</center>

**<center>Lucien DEROUET - Antoine PURIER</center>**

## Introduction

&emsp;&emsp;Dans le cadre de notre projet neo4j, nous construisons une base de données graphe à partir des metadonnées de publications hébergées sur le site https://paperswithcode.com. Des informations supplémentaires sur ces publications sont également récupérées sur d'autres sites via des API et du scraping.

Le but est ensuite de requêter cette base pour répondre à des questions sur le dataset, telles que:
- REMPLACER PAR REQUÊTE 1
- REMPLACER PAR REQUÊTE 2
- REMPLACER PAR REQUÊTE 3
- ...

## Les Données

&emsp;&emsp;Pour la récupération des données et la construction de notre dataset, nous partons du site https://paperswithcode.com. Son API publique nous permet facilement de récupérer les métadonnées des plus de 900.000 publications qu'il héberge.

&emsp;&emsp;Les metadonnées pour chaque papier sont stockées dans un dictionnaire qui est exporté au format json. Nous construirons notre BDD graphe à partir de ce fichier json d'entrée.

&emsp;&emsp;Ci-dessous un flowchart illustrant les différents briques mises en place pour la construction du dataset.

![title](flowchart.png)

Le code ayant servi à récupérer et à construire le dataset se trouve en annexe dans le dossier de rendu.

Dans la prochaine section, nous nous connectons à Neo4j et nous créons la base de donnée.

## Connexion à Neo4j

Nous commençons par vérifier la version de neo4j installée sur la machine.

In [1]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.4.1


Nous créons ensuite une classe pour la connexion à neo4j.

In [2]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Puis nous instantions une connexion.

In [3]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="antoine", pwd="easypassword")

## Création de la Base

Nous créons maintenant la base de données graphe `paperswithcode`.

In [7]:
conn.query("CREATE OR REPLACE DATABASE paperswithcode")

[]

Puis nous créons les noeuds et les relations du graphe à partir du fichier json.

Nous créons 6 noeuds:
- `Paper`
- `Author`
- `Publisher`
- `Organisation`
- `Conference`
- `Keyword`

Et nous relions ces noeuds à l'aide de 6 relations:
- Un auteur a écrit un papier : `WROTE`
- Un papier a été présenté à une conférence : `PRESENTED_AT`
- Un éditeur a publié un papier : `PUBLISHED`
- Un auteur est affilié à une organisation (université, entreprise, ...) : `AFFILIATED_TO`
- Un papier parle d'un sujet (évoque un mot clé) : `TALKS_ABOUT`
- Un papier fait référence à un autre papier : `REFERS_TO`

In [ ]:
query_string = '''
CALL apoc.load.json("file:dataset_18k_v7.json") YIELD value as paper
WITH paper, paper.authors as authors, paper.conference as conference, paper.references as references, paper.key_words as keywords
UNWIND authors as aut
UNWIND conference as conf
UNWIND references as ref
UNWIND keywords as key
MERGE (p:Paper {title: paper.title})
SET
    p.doi = paper.doi,
    p.language = paper.language,
    p.arxiv_category = paper.arxiv_category,
    p.publication_date = paper.date,
    p.references = references
MERGE (pub:Publisher {name: paper.publisher})
MERGE (a:Author {name: aut.name})
MERGE (o:Organisation {name:aut.organisation})
MERGE (c:Conference {name: conf.name})
MERGE (k:Keyword {name: key})
MERGE (a)-[:WROTE]->(p)
MERGE (p)-[:PRESENTED_AT]->(c)
MERGE (pub)-[:PUBLISHED]->(p)
MERGE (a)-[:AFFILIATED_TO]->(o)
MERGE (p)-[:TALKS_ABOUT]->(k)
'''
conn.query(query_string, db='paperswithcode')

In [6]:
query_string = '''
MATCH
  (p:Paper),
  (r:Paper)
WHERE r.doi IN p.references
CREATE (p)-[:REFERS_TO]->(r)
'''
conn.query(query_string, db='paperswithcode')

[]

## Requête de la Base

Maintenant que la base est créée et notre graphe modélisé, nous la requêtons.

In [60]:
import pandas as pd

In [79]:
labels = ["Paper", "Author", "Publisher", "Organisation", "Conference", "Keyword"]

d = {"Nodes Count": pd.NA}
df_res = pd.DataFrame(data=d, index=labels)
df_res

,Nodes Count
Paper,<NA>
Author,<NA>
Publisher,<NA>
Organisation,<NA>
Conference,<NA>
Keyword,<NA>


### Requête 1: Combien y-a-t-il de noeuds distincts pour chaque label?

In [108]:
labels = ["Paper", "Author", "Publisher", "Organisation", "Conference", "Keyword"]

d = {"Nombre de Noeuds": pd.NA}
df_res = pd.DataFrame(data=d, index=labels)

for label in labels:
    query_string = f'''
    MATCH (p:{label})
    RETURN count(*)
    '''
    res = [dict(_) for _ in conn.query(query_string, db='paperswithcode')]
    df_res.loc[label, "Nombre de Noeuds"] = res[0]["count(*)"]
    
display(df_res)   

,Nombre de Noeuds
Paper,3
Author,6
Publisher,2
Organisation,1
Conference,3
Keyword,7


### Requête 2: Quels sont les papiers les plus référencés?

In [115]:
query_string = '''
MATCH (p:Paper)-[rel:REFERS_TO]->(r:Paper)
RETURN r.title, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res.rename(columns={"r.title":"Titre du Papier", "count":"Nombre de Référencements"})

,Titre du Papier,Nombre de Référencements
0,An algebraic criterion of the Darboux integrab...,2
1,An Algebraic-Geometric Approach for Linear Reg...,1


### Requête 3: Quelles sont les organisations les plus représentées dans le dataset?

In [114]:
query_string = '''
MATCH (o:Organisation)
RETURN o.name, count(o) as count
ORDER BY count desc
LIMIT 3
'''

df_res = DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res.rename(columns={"o.name":"Organisation", "count":"Auteurs Affiliés"})

,Organisation,Auteurs Affiliés
0,Université de Technologie de Troyes,1


### Requête 4: Quels sont les mots clé les plus cités?

In [117]:
query_string = '''
MATCH (p:Paper)-[rel:TALKS_ABOUT]->(k:Keyword)
RETURN k.name, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res.rename(columns={"k.name":"Mot-clé", "count":"Nombre de Citations"})

,Mot-clé,Nombre de Citations
0,maths,2
1,Darboux,1
2,algebra,1
3,linear regression,1
4,equations,1


### Requête 5: Quel est le nombre de papiers publiés par publishers?

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 6: Existe-t-il des liens de référence entre certains papiers du dataset?

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 7

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 8

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 9

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 10

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

## Fermeture de la Connexion

In [ ]:
conn.close()